In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip uninstall transformers -Y
    %pip install transformers

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/mres/model_harvesting


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -Y
Mounted at /content/gdrive
/content/gdrive/MyDrive/mres/model_harvesting


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
from IPython.display import clear_output

In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def free_mem(vars):
    for v in vars: del v
    gc.collect()
    t.cuda.empty_cache()

In [4]:
models = {
    "llama2": "meta-llama/Llama-2-7b-chat-hf",
    "llama3": "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistral": "mistralai/Mistral-7B-v0.1"
}

model_name = "llama2"
model = AutoModelForCausalLM.from_pretrained(models[model_name], torch_dtype=t.float16).to(device); model.eval()
tokenizer = AutoTokenizer.from_pretrained(models[model_name])

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
n_layer, d_model = 32, 4096
aspects = ["coherence", "consistency", "fluency", "relevance"]

### newsroom

In [6]:
for choice in [1, 2]:
    path = f"prompts_short/newsroom-mine-compare-{choice}.jsonl"
    prompts = pd.read_json(path, orient="records", lines=True)
    for aspect in aspects:
        activations = t.zeros(len(prompts), d_model)
        for i in trange(len(prompts), desc=f"{aspect}-{choice}"):
            prompt = prompts.at[i, f"prompt-{aspect}"]
            tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
            with t.no_grad(): out = model(tks, output_hidden_states=True)
            activations[i] = out["hidden_states"][-1][0, -1, :].cpu()
            free_mem([tks, out])
        t.save(activations, f"{model_name}/activations_short/newsroom_{aspect}_{choice}.pt")
        free_mem([activations])

coherence-1:   0%|          | 0/1260 [00:00<?, ?it/s]

consistency-1:   0%|          | 0/1260 [00:00<?, ?it/s]

fluency-1:   0%|          | 0/1260 [00:00<?, ?it/s]

relevance-1:   0%|          | 0/1260 [00:00<?, ?it/s]

coherence-2:   0%|          | 0/1260 [00:00<?, ?it/s]

consistency-2:   0%|          | 0/1260 [00:00<?, ?it/s]

fluency-2:   0%|          | 0/1260 [00:00<?, ?it/s]

relevance-2:   0%|          | 0/1260 [00:00<?, ?it/s]

### summeval

In [7]:
for choice in [1, 2]:
    path = f"prompts_short/summeval-mine-compare-{choice}.jsonl"
    prompts = pd.read_json(path, orient="records", lines=True)
    for aspect in aspects:
        activations = t.zeros(len(prompts), d_model)
        for i in trange(len(prompts), desc=f"{aspect}-{choice}"):
            prompt = prompts.at[i, f"prompt-{aspect}"]
            tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
            with t.no_grad(): out = model(tks, output_hidden_states=True)
            activations[i] = out["hidden_states"][-1][0, -1, :].cpu()
            free_mem([tks, out])
        t.save(activations, f"{model_name}/activations_short/summeval_{aspect}_{choice}.pt")
        free_mem([activations])

coherence-1:   0%|          | 0/12000 [00:00<?, ?it/s]

consistency-1:   0%|          | 0/12000 [00:00<?, ?it/s]

fluency-1:   0%|          | 0/12000 [00:00<?, ?it/s]

relevance-1:   0%|          | 0/12000 [00:00<?, ?it/s]

coherence-2:   0%|          | 0/12000 [00:00<?, ?it/s]

consistency-2:   0%|          | 0/12000 [00:00<?, ?it/s]

fluency-2:   0%|          | 0/12000 [00:00<?, ?it/s]

relevance-2:   0%|          | 0/12000 [00:00<?, ?it/s]